In [ ]:
!pip install peft
!pip install datasets #necesary to load the Datasets.
! pip install -U accelerate
!pip install transformers[torch] -U
!pip install sentencepiece
!pip install sentencepiece --user
!pip install transformers[torch]

In [ ]:
!apt install transformers[torch]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package transformers[torch]


In [ ]:
!apt install transformers[torch]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package transformers[torch]


In [ ]:
!apt install -U accelerate

E: Command line option 'U' [from -U] is not understood in combination with the other options.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
model_name = "bigscience/bloomz-560m"
NUM_VIRTUAL_TOKENS = 4  # number of tokens we want to add to the prompt,  more tokens means more parameters to be trained.
NUM_EPOCHS = 5


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
# gpu_device = 0  # T4 GPU typically has index 0
# import tensorflow as tf

# # Set the visible devices
# tf.config.experimental.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[gpu_device], 'GPU')
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


An initial test with the pretrained model.

In [ ]:
def get_outputs(model, inputs, max_new_tokens=100, device=device):
    # Ensure inputs are on the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate outputs from the model
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_new_tokens,
            repetition_penalty=1.5,  # Avoid repetition.
            early_stopping=True,     # The model can stop before reaching the max_length
            eos_token_id=model.config.eos_token_id
        )

    return outputs


In [ ]:

#Inference original model
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt").to(device)
foundational_outputs_sentence = get_outputs(foundation_model, input_sentences, max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


["I want you to act as a motivational coach.  Don't be afraid of being challenged."]


In [ ]:

from datasets import load_dataset
dataset = "fka/awesome-chatgpt-prompts"

#Create the Dataset to create prompts.

data = load_dataset(dataset)
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
train_sample = data["train"]

train_sample = train_sample.remove_columns('act')

display(train_sample)


Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 153
})

In [ ]:
dataset_sentences = load_dataset("Abirate/english_quotes")

data_sentences = dataset_sentences.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample_sentences = data_sentences["train"]
train_sample_sentences = train_sample_sentences.remove_columns(['author', 'tags'])
display(train_sample_sentences)


Dataset({
    features: ['quote', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
import peft

from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=4, #As bigger the R bigger the parameters to train.
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"], #You can obtain a list of target modules in the URL above.
    lora_dropout=0.05, #Helps to avoid Overfitting.
    bias="lora_only", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)


In [ ]:
peft_model = get_peft_model(foundation_model, lora_config).to(device)
# print(peft_model.print_trainable_parameters())


In [ ]:
#Create a directory to contain the Model
import os
working_dir = './'

output_directory = os.path.join(working_dir, "peft_lab_outputs")


In [ ]:
#Creating the TrainingArgs
import transformers
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=output_directory,
    auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=2,
    use_cpu=False if device.type == 'cuda' else True
)


In [ ]:
#This cell may take up to 15 minutes to execute.
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=78, training_loss=4.219196026141827, metrics={'train_runtime': 30.6933, 'train_samples_per_second': 9.97, 'train_steps_per_second': 2.541, 'total_flos': 79952706674688.0, 'train_loss': 4.219196026141827, 'epoch': 2.0})

In [ ]:
peft_model_path = os.path.join(output_directory, f"lora_model")

trainer.model.save_pretrained(peft_model_path)

In [ ]:
loaded_model = PeftModel.from_pretrained(foundation_model,
                                        peft_model_path,
                                        is_trainable=False)

In [ ]:
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt")
foundational_outputs_sentence = get_outputs(loaded_model, input_sentences, max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['I want you to act as a motivational coach.  I will provide advice on how best approach is needed for achieving my goals and making sure that the goal of this program has been achieved. My first request is: "I need help developing an effective, focused']
